In [1]:
import os


In [2]:
%pwd

'f:\\Files\\DS&ML\\Flight-Fare-Price-Prediction\\Exp'

In [3]:
os.chdir('../')

In [4]:
%pwd

'f:\\Files\\DS&ML\\Flight-Fare-Price-Prediction'

In [5]:
from dataclasses import dataclass
from pathlib import Path

In [6]:
@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    model_path: Path
    test_raw_data: Path
    all_params: dict
    metric_file_path: Path
    preprocessor_path: Path
    target_column: str

In [7]:
from mlproject.utils.common import *
from mlproject.constants import *

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH,
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config["artifacts_root"]])


    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.GradientBoostingRegressor
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_raw_data=config.test_raw_data,
            model_path=config.model_path,
            all_params=params,
            metric_file_path=config.metric_file_path,
            preprocessor_path=config.preprocessor_path,
            target_column=schema.name,
        )
        return model_evaluation_config

In [9]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pandas as pd
import joblib
import json
import mlflow
import dagshub


In [10]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

        # Initialize MLflow tracking
        os.environ['MLFLOW_TRACKING_USERNAME'] = "JavithNaseem-J"
        os.environ['MLFLOW_TRACKING_PASSWORD'] = "f4e2b4e74fa3a3ae5b91b9b918dab74745cecd30"
        
        dagshub.init(repo_owner="JavithNaseem-J", repo_name="Flight-Fare-Price-Prediction")
        mlflow.set_tracking_uri("https://dagshub.com/JavithNaseem-J/Flight-Fare-Price-Prediction.mlflow")
        mlflow.set_experiment("Flight-Fare-Price-Prediction")

    def evaluate(self):
        try:
            if not Path(self.config.test_raw_data).exists():
                raise FileNotFoundError(f"Test data file not found at {self.config.test_raw_data}")
            if not Path(self.config.preprocessor_path).exists():
                raise FileNotFoundError(f"Preprocessor file not found at {self.config.preprocessor_path}")
            if not Path(self.config.model_path).exists():
                raise FileNotFoundError(f"Model file not found at {self.config.model_path}")

            with mlflow.start_run():
                # Set tags for the run
                mlflow.set_tag("model_type", "GradientBoostingRegressor")
                mlflow.set_tag("evaluation_stage", "testing")

                logger.info("Loading preprocessor and model...")
                preprocessor = joblib.load(self.config.preprocessor_path)
                model = joblib.load(self.config.model_path)

                # Log model parameters
                mlflow.log_params(self.config.all_params)

                logger.info(f"Loading test data from {self.config.test_raw_data}...")
                test_data = pd.read_csv(self.config.test_raw_data)
                
                # Normalize column names to lowercase
                test_data.columns = test_data.columns
                target_column = self.config.target_column

                # Extract target column
                if target_column not in test_data.columns:
                    raise KeyError(f"Target column '{target_column}' not found in test data. Available columns: {list(test_data.columns)}")

                test_y = test_data[target_column]
                test_x = test_data.drop(columns=[target_column])

                logger.info(f"Test data shape: X={test_x.shape}, y={test_y.shape}")

                logger.info("Preprocessing test features...")
                test_x_transformed = preprocessor.transform(test_x)

                logger.info("Making predictions on the test data...")
                predictions = model.predict(test_x_transformed)

                logger.info("Evaluating model performance...")
                mse = mean_squared_error(test_y, predictions)
                mae = mean_absolute_error(test_y, predictions)
                r2 = r2_score(test_y, predictions)
                

                # Create metrics dictionary
                metrics = {
                    "mean_squared_error": mse,
                    "mean_absolute_error": mae,
                    "r2_score": r2
                }
                
                logger.info(metrics)
                
                # Log metrics to MLflow
                mlflow.log_metrics(metrics)

                # Log model with signature
                signature = mlflow.models.infer_signature(
                    test_x_transformed, predictions
                )
                mlflow.sklearn.log_model(
                    model,
                    "flight_fare_model",  
                    signature=signature,
                    registered_model_name="Flight Fare Prediction"
                )

                # Save metrics
                metrics_file = Path(self.config.root_dir) / "metrics.json"
                with open(metrics_file, "w") as f:
                    json.dump(metrics, f, indent=4)
                    
                logger.info(f"Evaluation metrics saved at {metrics_file}")

                return metrics

        except Exception as e:
            logger.error(f"Error in model evaluation: {str(e)}")
            raise e


In [11]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    metrics = model_evaluation.evaluate()

except FileNotFoundError as e:
    logger.error(f"File not found: {e}")
except KeyError as e:
    logger.error(f"Missing key in configuration or data: {e}")
except Exception as e:
    logger.error(f"Unexpected error: {e}")

[2025-04-21 23:04:38,172: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-04-21 23:04:38,173: INFO: common: yaml file: params.yaml loaded successfully]
[2025-04-21 23:04:38,185: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-04-21 23:04:38,188: INFO: common: created directory at: artifacts]
[2025-04-21 23:04:38,189: INFO: common: created directory at: artifacts/model_evaluation]
[2025-04-21 23:04:38,603: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]


Accessing as JavithNaseem-J

[2025-04-21 23:04:38,619: INFO: helpers: Accessing as JavithNaseem-J]
[2025-04-21 23:04:39,015: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/repos/JavithNaseem-J/Flight-Fare-Price-Prediction "HTTP/1.1 200 OK"]
[2025-04-21 23:04:39,405: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]


Initialized MLflow to track repo "JavithNaseem-J/Flight-Fare-Price-Prediction"

[2025-04-21 23:04:39,412: INFO: helpers: Initialized MLflow to track repo "JavithNaseem-J/Flight-Fare-Price-Prediction"]


...

[2025-04-21 23:04:39,418: INFO: helpers: Repository JavithNaseem-J/Flight-Fare-Price-Prediction initialized!]
[2025-04-21 23:04:41,242: INFO: 1784283822: Loading preprocessor and model...]
[2025-04-21 23:04:41,885: INFO: 1784283822: Loading test data from artifacts/data_transformation/test.csv...]
[2025-04-21 23:04:41,917: INFO: 1784283822: Test data shape: X=(14250, 9), y=(14250,)]
[2025-04-21 23:04:41,917: INFO: 1784283822: Preprocessing test features...]
[2025-04-21 23:04:41,924: INFO: 1784283822: Making predictions on the test data...]
[2025-04-21 23:04:41,969: INFO: 1784283822: Evaluating model performance...]
[2025-04-21 23:04:41,973: INFO: 1784283822: {'mean_squared_error': 0.2193376383129778, 'mean_absolute_error': 0.3601831432532933, 'r2_score': 0.8895826111244571}]


Registered model 'Flight Fare Prediction' already exists. Creating a new version of this model...
2025/04/21 23:04:57 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Flight Fare Prediction, version 24


[2025-04-21 23:04:57,098: INFO: 1784283822: Evaluation metrics saved at artifacts\model_evaluation\metrics.json]


Created version '24' of model 'Flight Fare Prediction'.


🏃 View run intelligent-slug-48 at: https://dagshub.com/JavithNaseem-J/Flight-Fare-Price-Prediction.mlflow/#/experiments/0/runs/7047cf4ee4d04082a789e7f6876f1fe7
🧪 View experiment at: https://dagshub.com/JavithNaseem-J/Flight-Fare-Price-Prediction.mlflow/#/experiments/0
